In [1]:
from jinja2 import Template, Environment, FileSystemLoader
import os
import json

env = Environment(loader=FileSystemLoader("./templates"))
#template = env.get_template("index.tmpl")
#print(template.render())

# 0. Retrieving the raw json file for profiles

In [2]:
!wget https://github.com/BioSchemas/bioschemas-dde/raw/main/bioschemas.json 

--2025-02-18 11:38:21--  https://github.com/BioSchemas/bioschemas-dde/raw/main/bioschemas.json
Résolution de github.com (github.com)… 140.82.121.3
Connexion à github.com (github.com)|140.82.121.3|:443… connecté.
requête HTTP transmise, en attente de la réponse… 302 Found
Emplacement : https://raw.githubusercontent.com/BioSchemas/bioschemas-dde/main/bioschemas.json [suivant]
--2025-02-18 11:38:21--  https://raw.githubusercontent.com/BioSchemas/bioschemas-dde/main/bioschemas.json
Résolution de raw.githubusercontent.com (raw.githubusercontent.com)… 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connexion à raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 305680 (299K) [text/plain]
Sauvegarde en : « bioschemas.json.2 »

bioschemas.json.2   100%[===================>] 298,52K  --.-KB/s    ds 0,03s   

2025-02-18 11:38:21 (10,5 MB/s) — « bioschemas.json.2 » sauvegardé [305680/305680]

# 1. Generating a single markdown document as an index of profiles 

In [3]:
cross_domain_profiles = ["ComputationalTool", "ComputationalWorkflow", "FormalParameter", 
                         "Dataset", "DataCatalog", "Course", "CourseInstance", "TrainingMaterial"]

In [4]:
with open("./bioschemas.json") as file:
    profiles_dump = json.load(file)

In [5]:
#profiles_dump

In [6]:
def list_profiles(data):
    profiles = []
    for p in data["@graph"]:
        print("###")
        print(p["rdfs:label"])
        print()
        print(p["rdfs:comment"])
        print()
        name = p["rdfs:label"]
        desc = p["rdfs:comment"]
        if name in cross_domain_profiles:
            #[a relative link](other_file.md)
            profiles.append({"name": f"[{name}](profiles/{name}.md)", 
                             "description": desc})
    return profiles

In [7]:
all_profiles = list_profiles(profiles_dump)

###
ChemicalSubstance

This profile describes a ChemicalSubstance which is 'a portion of matter of constant composition, composed of molecular entities of the same type or of different types' (source: ChEBI:59999). Note, the parent class for this profile has been updated to a pending class in schema.org.

###
Gene

Bioschemas profile describing a Gene in Life Sciences. This Gene profile specification presents the markup for describing a Gene and is based on the 1.0 release. Note, the parent class for this profile has been updated to a pending class in schema.org.

###
Taxon

Bioschemas profile for describing a biological taxon This profile aims to denote a taxon by common properties such as its scientific name, taxonomic rank and vernacular names. It is also a means to link to existing taxonomic registers where each taxon has a URI. Version 1.0-RELEASE.<br/><h3>Summary of Changes</h3><br>Changes since 0.8-DRAFT:<ul><li>None. Version 0.8-DRAFT was the draft approved for release.</li></u

In [55]:
#profiles
#print(json.dumps(all_profiles, indent=True))

In [8]:
profiles_tmpl = env.get_template("profiles.tmpl")
output = profiles_tmpl.render(profiles=all_profiles)
print(output)

# Welcome to schemas.science

## What is schemas.science?

**[schemas.science](https://schemas.science)** is a community effort which aims at improving the Findability on the Web
of scientific research data, products and resources. It does this by encouraging the use
of [Schema.org](https://schema.org/) markup in their
websites so that they are indexable by search engines and other services. schemas.science encourages the consistent use
of markup tailored for diverse scientific domains. This structured information then makes it easier to
discover, collate, analyse and consume distributed resources.

## Cross domain schemas


| Name              | Description                          |
| :---------------- | :----------------------------------- |
| [FormalParameter](profiles/FormalParameter.md) | Bioschemas specification for describing a formal parameter in the Life Sciences. Version: 1.0-RELEASE (09 March 2021)   |
| [ComputationalTool](profiles/ComputationalTool.md) | The Life Science 

In [9]:
with open("../docs/index.md", 'w') as out_file:
    out_file.writelines(output)